In [34]:
import pandas as pd
import os

# Create directory for LaTeX tables
os.makedirs('latex_tables', exist_ok=True)

df = pd.read_csv('analysis/results/patient_differences.csv')

# Reshape data for LaTeX table
# We want a table with patients as rows, and columns showing RMSE for each approach/horizon combination
pivot_mean = df.pivot_table(
    index='Patient',
    columns=['Approach', 'Prediction Horizon'],
    values='mean'
)

pivot_std = df.pivot_table(
    index='Patient',
    columns=['Approach', 'Prediction Horizon'],
    values='std'
)

# Load the raw data to calculate overall RMSE properly
raw_df = pd.read_csv('analysis/evaluation_metrics.csv')

# Calculate average RMSE and std for each approach and prediction horizon
# Instead of averaging the patient-level RMSEs, calculate RMSE directly from all data points
avg_values = {}
std_values = {}
for approach in ['pixtral-large-latest', 'nollm']:
    for horizon in [6, 9, 12, 18, 24]:
        # Filter data for this approach and horizon
        filtered_data = raw_df[(raw_df['Approach'] == approach) & 
                              (raw_df['Prediction Horizon'] == horizon)]
        # Calculate RMSE directly from all data points
        avg_values[(approach, horizon)] = filtered_data['RMSE'].mean()
        std_values[(approach, horizon)] = filtered_data['RMSE'].std()

# Format the LaTeX table with nice styling
styled_latex_template = """\\begin{table}[ht]
\\centering
\\caption{RMSE by Patient, Approach, and Prediction Horizon (mg/dL)}
\\label{tab:performance_table}
\\renewcommand{\\arraystretch}{1.3}
\\setlength{\\tabcolsep}{6pt}
\\resizebox{\\textwidth}{!}{%%
\\begin{tabular}{|l|%s|}
\\hline
\\rowcolor{gray!25} \\multirow{2}{*}[1ex]{\\textbf{Patient}} & %s \\\\
\\rowcolor{gray!25} & %s \\\\
\\hline
%s
\\hline
\\rowcolor{blue!10} \\textbf{Average} & %s \\\\
\\hline
\\end{tabular}
}
\\end{table}
"""

# Create column headers
approaches = ['pixtral-large-latest', 'nollm']
horizons = [6, 9, 12, 18, 24]

# Format headers for LaTeX
column_format = "|".join(["cc" for _ in horizons])
header_parts = []
for h in horizons:
    header_parts.append("\\multicolumn{2}{c|}{\\cellcolor{gray!25}\\textbf{Horizon " + str(h*5) + " Min" + "}}")
header_row = " & ".join(header_parts)
subheader_parts = []
for _ in horizons:
    subheader_parts.append("\\cellcolor{green!10}{\\textbf{With}} & \\cellcolor{orange!10}{\\textbf{Without}}")
subheader_row = " & ".join(subheader_parts)

# Format data rows with background colors and bold for better values
rows = []
for patient in pivot_mean.index:
    row_data = []
    for horizon in horizons:
        # Get RMSE and std for both approaches at this horizon
        with_mean = pivot_mean.loc[patient, ('pixtral-large-latest', horizon)]
        without_mean = pivot_mean.loc[patient, ('nollm', horizon)]
        with_std = pivot_std.loc[patient, ('pixtral-large-latest', horizon)]
        without_std = pivot_std.loc[patient, ('nollm', horizon)]
        
        # Format with 2 decimal places, include std dev, and bold the better value
        if with_mean < without_mean:
            cell_with = f"\\cellcolor{{green!10}}{{\\textbf{{{with_mean:.2f} $\\pm$ {with_std:.2f}}}}}"
            cell_without = f"\\cellcolor{{orange!10}}{{{without_mean:.2f} $\\pm$ {without_std:.2f}}}"
        else:
            cell_with = f"\\cellcolor{{green!10}}{{{with_mean:.2f} $\\pm$ {with_std:.2f}}}"
            cell_without = f"\\cellcolor{{orange!10}}{{\\textbf{{{without_mean:.2f} $\\pm$ {without_std:.2f}}}}}"
        row_data.append(f"{cell_with} & {cell_without}")
    
    # Join the data for this patient with alternating row colors
    row_color = "gray!5" if int(patient) % 2 == 0 else "white"
    rows.append(f"\\rowcolor{{{row_color}}} {patient} & {' & '.join(row_data)} \\\\")

# Combine all rows
all_rows = "\n".join(rows)

# Format average row
avg_row_data = []
for horizon in horizons:
    # Get average RMSE and std for both approaches at this horizon
    with_mean = avg_values[('pixtral-large-latest', horizon)]
    without_mean = avg_values[('nollm', horizon)]
    with_std = std_values[('pixtral-large-latest', horizon)]
    without_std = std_values[('nollm', horizon)]
    
    # Format with 2 decimal places, include std dev, and bold the better value
    if with_mean < without_mean:
        cell_with = f"\\cellcolor{{green!10}}{{\\textbf{{{with_mean:.2f} $\\pm$ {with_std:.2f}}}}}"
        cell_without = f"\\cellcolor{{orange!10}}{{{without_mean:.2f} $\\pm$ {without_std:.2f}}}"
    else:
        cell_with = f"\\cellcolor{{green!10}}{{{with_mean:.2f} $\\pm$ {with_std:.2f}}}"
        cell_without = f"\\cellcolor{{orange!10}}{{\\textbf{{{without_mean:.2f} $\\pm$ {without_std:.2f}}}}}"
    avg_row_data.append(f"{cell_with} & {cell_without}")

avg_row = " & ".join(avg_row_data)

# Generate the full LaTeX table
styled_latex_table = styled_latex_template % (column_format, header_row, subheader_row, all_rows, avg_row)

# Write to file
with open('latex_tables/performance_table.tex', 'w') as f:
    f.write(styled_latex_table)

In [30]:
df = pd.read_csv('analysis/evaluation_metrics.csv')
df['Patient'] = df['Patient'].astype(str)
agg_data = df.groupby(['Approach', 'Prediction Horizon'])['RMSE'].agg(['mean', 'std']).reset_index()

# Create a DataFrame to store differences
diff_data = pd.DataFrame()

# Based on the LaTeX table, we need to use these approach names
approach1 = 'pixtral-large-latest'  # With meal features
approach2 = 'nollm'                 # Without meal features

for ph in sorted(agg_data['Prediction Horizon'].unique()):
    # Get mean values for each approach at this prediction horizon
    # Use .iloc[0] instead of .values[0] with proper error handling
    approach1_data = agg_data[(agg_data['Approach'] == approach1) & 
                    (agg_data['Prediction Horizon'] == ph)]
    approach2_data = agg_data[(agg_data['Approach'] == approach2) & 
                    (agg_data['Prediction Horizon'] == ph)]
        
    mean1 = approach1_data['mean'].iloc[0]
    mean2 = approach2_data['mean'].iloc[0]
    
    # Calculate absolute and percentage differences
    abs_diff = mean2 - mean1  # positive means approach1 is better (lower RMSE)
    pct_diff = (abs_diff / mean1) * 100  # percentage improvement
    
    # Store in dataframe (using concat instead of deprecated append)
    new_row = pd.DataFrame({
        'Prediction Horizon': [ph],
        'Absolute Difference': [abs_diff],
        'Percentage Difference': [pct_diff]
    })
    diff_data = pd.concat([diff_data, new_row], ignore_index=True)

# Find the prediction horizon with the maximum difference (if data exists)
if not diff_data.empty:
    max_abs_diff_ph = diff_data.loc[diff_data['Absolute Difference'].idxmax()]['Prediction Horizon']
    max_pct_diff_ph = diff_data.loc[diff_data['Percentage Difference'].idxmax()]['Prediction Horizon']
else:
    max_abs_diff_ph = None
    max_pct_diff_ph = None

print(f"Differences between {approach1} and {approach2}:")
print(diff_data)
print(f"\nMaximum absolute difference: {diff_data['Absolute Difference'].max():.2f} at PH = {max_abs_diff_ph}")
print(f"Maximum percentage difference: {diff_data['Percentage Difference'].max():.2f}% at PH = {max_pct_diff_ph}")

Differences between pixtral-large-latest and nollm:
   Prediction Horizon  Absolute Difference  Percentage Difference
0                   6             0.268654               1.643274
1                   9             0.698829               2.742420
2                  12             1.628740               4.928990
3                  18             2.128312               4.661503
4                  24             2.778841               4.949424

Maximum absolute difference: 2.78 at PH = 24.0
Maximum percentage difference: 4.95% at PH = 24.0
